## Import libraries and customize matplotlib styles params

In [1]:
# ipywidgets references
# https://matplotlib.org/ipympl/examples/full-example.html#changing-a-line-plot-with-a-slider
# https://ipywidgets.readthedocs.io/en/latest/how-to/index.html

import sys
IN_COLAB = 'google.colab' in sys.modules
continuous_update_sliders = True
if IN_COLAB:
   continuous_update_sliders = False
   from google.colab import output
   output.enable_custom_widget_manager()
   !pip install --quiet ipympl


import ipywidgets as widgets
from   ipywidgets import  GridspecLayout,VBox,HBox,Layout

import matplotlib.pyplot as plt
import numpy as np

plt.rcParams.update(plt.rcParamsDefault)
px2inch = 1/plt.rcParams['figure.dpi']

%matplotlib widget

plt.ioff();

def plot_sets(ax,gr=False,ti='',xla='',yla='',xli=False, yli=False,ticks_off=False,xticks_off=False,yticks_off=False, ba=False, ar=False,leg=False):
    
    ax.grid(gr)
    ax.set_title(ti)
    ax.set_xlabel(xla)
    ax.set_ylabel(yla)

    if   ticks_off==1: ax.set_xticklabels([]); ax.set_xticks([]);ax.set_yticklabels([]); ax.set_yticks([])
    elif ticks_off==2: ax.set_xticklabels([]); ax.set_xticks([])
    elif ticks_off==2: ax.set_yticklabels([]); ax.set_yticks([])

    if ba: ax.set_box_aspect(ba)
    if ar: ax.set_aspect(ar)

    if xli: ax.set_xlim(xli)
    if yli: ax.set_ylim(yli)

    if leg: ax.legend()


SMALL_SIZE  = 8
MEDIUM_SIZE = 10

# title
plt.rc('axes',titlesize=MEDIUM_SIZE,titleweight='bold')
# xy-labells
plt.rc('axes',labelsize=SMALL_SIZE)
# xy-ticks
plt.rc('xtick',labelsize=SMALL_SIZE)
plt.rc('ytick',labelsize=SMALL_SIZE)
plt.rc('legend',fontsize=SMALL_SIZE, framealpha=0.99) 


## Main Functions

In [2]:

def spectrum_frequencies(y,fs):
    N         = y.size
    Y         = np.fft.fft(y) 
    Yall      = np.abs(Y)/N
    Ypos      = np.copy( Yall[:int(N/2)+1] )
    Ypos[1:] *= 2

    fall = np.arange(0,N)*fs/N
    fpos = np.linspace(0,fs/2,int(N/2)+1)

    return fpos,Ypos

def init_values(data):
    data_updated = []
    for di in data:
        x = (di['max']-di['min']) * np.random.rand(3)  + di['min']
        di['values_arr'] = np.round(x/di['step'])*di['step']
        data_updated.append(di)
    return  data_updated 

def get_noise(t,A_noise):
    N   = t.size
    return A_noise*( 2*np.random.rand(N)-1 )

def get_signals(t,fs,grid,grid2,nsignals):

    # signals[0] = signal0 
    # signals[1] = signal1
    # signals[2] = signal2
    # signals[3] = sum_signal = signal(0+1+2)
    # signals[4] = DFT(sum_signal)

    signals = []
    A_noise = grid2[0,0].value
    DC      = grid2[1,0].value
    sum     = DC*np.ones_like(t) + get_noise(t,A_noise)
    for i in range(nsignals):
        A, f, p  = grid[0,i].value,grid[1,i].value,grid[2,i].value
        signal_i = A*np.sin(2*np.pi*t*f + p*np.pi/180)
        sum     += signal_i
        signals.append(signal_i)
    signals.append(sum)
    
    # DFT
    _,Ypos = spectrum_frequencies(signals[-1],fs)
    signals.append(Ypos)
    
    return signals



## Demo

In [3]:

plt.close('all')

# Init data for signals
data = [{'description':'A  [V]', 'min':1,   'max':10, 'step':1   }, # Amplitudes
        {'description':'f  [Hz]','min':0.5, 'max':10, 'step':0.5 }, # Frequencies
        {'description':'ph [°]', 'min':-90, 'max':90, 'step':5    }] # Phases
data = init_values(data)

# Init data for sum of signals
data2 = [{'description':'Noise [V]', 'min':0, 'max':10,'step':0.25  }, 
         {'description':'DC [V]',    'min':0, 'max':10,'step':1.00  }] 

# Figure plt Layout
layout = [[0,0,1,1,2,2],
          [3,3,3,3,4,4]]

fig, ax = plt.subplot_mosaic(layout,constrained_layout=True)
fig.canvas.toolbar_position = 'right'
fig.canvas.header_visible   = False
kaxis   = 1.1
lw      = 1

# Time and frequency vectors
N   = 1000                       # Npoints            
fs  = 100                        # Sampling rate Hz
Ts  = 1/fs
t   = np.arange(N) * Ts          # Time vector
f,_ = spectrum_frequencies(t,fs) # Frequency vector

nsignals   = 3
ampl_max   = data[0]['max']
frec_max   = data[1]['max']
ncycles    = 20
xlims_time = [0,ncycles*frec_max**-1]
xlims_frec = [-0.5,fs/8]
lines      = []
#colors     = ['tab:blue','tab:orange','tab:green','tab:red','tab:purple']
colors     = ['#1f77b4' ,'#ff7f0e'   ,'#2ca02c'  ,'#d62728','#9467bd'    ]
sliders_width = '250px'

def update_plots(change):

    signals = get_signals(t,fs,grid,grid2,nsignals)
    for i,(line,signal) in enumerate(zip(lines,signals)):

        # plot signals
        if i<nsignals:
           line[0].set_data(t,signal)
        # plot signal sum
        elif i==nsignals:
           line[0].set_data(t,signal)
           y_abs_max = np.max(np.abs(signal))*kaxis
           ax[i].set_ylim(-y_abs_max,y_abs_max)
        # plot DFT of sum
        elif i==nsignals+1:
           line[0].set_data(f,signal)
           ax[i].set_ylim(0,np.max(signal)*kaxis)
    fig.canvas.draw()


def create_sliders_grid(nsignals,data):
    
    grid = GridspecLayout(nsignals,nsignals,width='auto',grid_gap='1px')
    for i in range(nsignals**2):
                row,col        = i//nsignals, i%nsignals
                di             = data[row]
                grid[row, col] = widgets.FloatSlider(**di,value=di['values_arr'][col],continuous_update=continuous_update_sliders)
                grid[row, col].style.handle_color = colors[col]
                grid[row, col].layout.width       = sliders_width
                grid[row, col].layout.margin      = '0px 0px 0px 0px'
                grid[row, col].layout.padding     = '0px 0px 0px 0px'
                grid[row, col].observe(update_plots)
    return grid

def create_sliders_grid2(data2):
    
    grid2 = GridspecLayout(2,1)
    for i,di in enumerate(data2):
        grid2[i,0] = widgets.FloatSlider(**di,continuous_update=continuous_update_sliders)
        grid2[i,0].layout.width=sliders_width
        grid2[i,0].style.handle_color = colors[3]
        grid2[i,0].observe(update_plots)
    return grid2


def init_plots(init_signals):

    for i,signal in enumerate(init_signals):

        # plot signals
        if i<nsignals:
            lines.append( ax[i].plot(t,signal,color=colors[i],lw=lw ) )
            plot_sets(ax[i], ti=f'$y_{i}$',xla='Time [s]',xli=xlims_time, yli=[-ampl_max*kaxis,ampl_max*kaxis])
            
        # plot signal sum
        elif i==nsignals: 
            lines.append( ax[i].plot(t,signal,color=colors[i],lw=lw) ) 
            y_abs_max = np.max(np.abs(signal))*kaxis
            plot_sets(ax[i], xli=[0,4],xla='Time [s]',yli=[-y_abs_max,y_abs_max],ti=f'$y_3=y_0 + y_1 + y_2 + DC + Noise$')

        # plot DFT of sum
        elif i==nsignals+1: 
            lines.append( ax[i].plot(f,signal,color=colors[i],lw=lw) )
            plot_sets(ax[i],gr=True,ti='$|DFT(y_3)|$', xli=xlims_frec, yli=[0,np.max(signal)*kaxis], xla='Frequency [Hz]')
            ax[i].yaxis.tick_right()
            ax[i].set_xticks(np.arange(0,14,2))
            

def random_case_single(change):

    i   = np.random.randint(0,nsignals**2)
    row = i // nsignals
    col = i %  nsignals
    di  = data[row]
    x   = (di['max']-di['min']) * np.random.rand()  + di['min']
    xr  = np.round(x/di['step'])*di['step']
    grid[row, col].value = xr


button = widgets.Button(description="Random Case",layout=Layout(border ='1px solid black'))
button.on_click(random_case_single)

grid         = create_sliders_grid(nsignals,data)
grid2        = create_sliders_grid2(data2)
init_signals = get_signals(t,fs,grid,grid2,nsignals)
init_plots(init_signals)


# # Create a centered layout with ipywidgets
layout = Layout(display='flex',
                        justify_content='center',
                        align_items='center',
                        width='auto',
                        border ='0px solid black', 
                        padding='0px')  # Adjust width as needed

#Arrange widgets in a Hbox and VBox
hbox = HBox([grid2,button],         layout=layout)
vbox = VBox([grid,hbox,fig.canvas], layout=layout)

# Display the layout
display(vbox)

def random_case_all(data,data2):
    
    # Signals sliders
    new_data = init_values(data)
    for row in range(nsignals):
            arr = new_data[row]['values_arr']
            for col in range(nsignals):
                #grid[row, col].unobserve(update_plots)
                grid[row, col].value = arr[col]
                
   # Noise DC sliders
    for i,di in enumerate(data2):
        k     = 4
        x     = (di['max']/4-di['min']) * np.random.rand()  + di['min']
        xround = np.round(x/di['step'])*di['step']
        grid2[i,0].value = xround

